https://github.com/ShenQingchao/QAQA/tree/master

In [ ]:
#!git clone https://github.com/ShenQingchao/QAQA

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
#exit()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install transformers
!pip install sentence_transformers

In [ ]:
#!apt-get install python3.7

In [ ]:
!pip install spacy

In [ ]:
!pip install benepar==0.2.0
!pip install pattern==3.6

In [ ]:
#!pip install neuralcoref==4.0

In [ ]:
#from QAUtil.qa import QA, QAList

def load_qa(path, end_id=1363, remove_redundancy_context=False):
    #data = pd.read_csv(path, sep='\t', header=None, names=['q_c', 'a'])
    data = pd.read_csv(path, encoding='utf-8')

    qa_list_obj = QAList()
    before_context = ""
    print("Total data "+str(len(data)))
    for i in range(len(data)):
        if i >= end_id:  # the max length of the dev-set is squad2_dev(11873)
            break

        uid = data['uid'][i]
        question = data['question'][i]
        answer = data['answer'][i]
        title = data['so_question_title'][i]
        context = 'Question: '+title + '\n' +data['so_question_body'][i]+ '\nAnswer: ' + data['so_answer'][i]

        # question_context = data['q_c'][i]
        # question = question_context.split("\\n")[0]
        # context_title = question_context.split("\\n")[1]
        # # print(i, question_context, context_title)
        # title, context = extract_title_from_context(context_title)
        if remove_redundancy_context and before_context == context:
            # print("redundant context...")
            continue
        qa = QA(question, answer, context, title, uid)
        qa_list_obj.append_(qa)
        before_context = context
    logger.debug(f"qa number: {len(data)}")
    qa_list_obj.set_all_context_embedding()
    return qa_list_obj


In [ ]:
from sentence_transformers import util
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-stsb-mean-tokens')  # ../3rd_models/multi-qa-MiniLM-L6-cos-v1

def is_same_answer(ans1, ans2, is_bool=False):
    ans1 = ans1.strip().lower()
    ans2 = ans2.strip().lower()

    original_embedding = sbert_model.encode(ans1)
    compare_embedding = sbert_model.encode(ans2)

    similarity = util.cos_sim(original_embedding, compare_embedding).tolist()[0]

    if similarity[0] >= 0.76:
        return True
    else:
        return False


def calculate_sim_origin_sentence(origin, target_list, top_n):
    '''
    This method can calculate the similarity between origin and target.
    :param origin:  a word or a sentence.
    :param target_list:  a list containing words or sentences.
    :param top_n:  the number of return list.
    :param context_embedding: the embedding of target_list.
    :return: 2-dim list  --- the top_n most similar target words/sentences for origin; e.g., [('dog', 0.9), ('cat', 0.85)]
    '''
    sim_dict = {}
    # model = SentenceTransformer('../pretrained-models/multi-qa-MiniLM-L6-cos-v1')   # bert-base-nli-stsb-mean-tokens , multi-qa-MiniLM-L6-cos-v1, paraphrase-MiniLM-L6-v2
    w_embedding = sbert_model.encode(origin)

    context_embedding = sbert_model.encode(target_list)
    sim = util.cos_sim(w_embedding, context_embedding).tolist()[0]
    # dict: {sent: sim_value}: first, load sent and score to a dict; and then rank by score--> convert to list
    for i, score in enumerate(sim):
        sim_dict[target_list[i]] = score
    res_context_score_list = sorted(sim_dict.items(), key=lambda x: x[1], reverse=True)  # dict to 2-dim list
    if not top_n:
        top_n = len(target_list)
    return res_context_score_list[:top_n]


def calculate_sim_origin_target(origin, target_obj, top_n):
    sim_dict = {}
    c_embedding = target_obj.get_all_context_embedding()
    print("Embedding count "+str(len(c_embedding)))
    # contexts_embedding = target_obj.get_all_context_embedding()
    # contexts = list(contexts_embedding[:, 0])
    # c_embedding = list(contexts_embedding[:, 1])
    # model = SentenceTransformer('../pretrained-models/multi-qa-MiniLM-L6-cos-v1')   # bert-base-nli-stsb-mean-tokens , multi-qa-MiniLM-L6-cos-v1, paraphrase-MiniLM-L6-v2
    w_embedding = sbert_model.encode(origin)
    # sim = util.dot_score(w_embedding, c_embedding).tolist()[0]
    sim = util.cos_sim(w_embedding, c_embedding).tolist()[0]

    # return dict, <id, sim_value> # first: load qa_id and score to a dict, and then rank by score
    for i, score in enumerate(sim):
        sim_dict[i] = score
    res_context_score_list = sorted(sim_dict.items(), key=lambda x: x[1], reverse=True)
    # for item in res_context_score_list[:top_n]:  # only print the top_n related context
    #     print(item[1], item[0])
    if not top_n:
        top_n = len(c_embedding)
    return res_context_score_list[:top_n]



In [ ]:
import sys
sys.path.append('/QAQA-master/scripts') # update location
sys.path.append('/benepar_en3/') # update location

from utils.my_logger import logger
from utils.preprocess import word_embedding
import numpy as np


class QA:
    count = 0
    def __init__(self, q, a, c, t, uid):
        self.id = QA.count  # identify the line number of the qa
        QA.count += 1
        self.uid = uid
        self.Q = q
        self.A = a
        # self.C = context_preprocess(c)
        self.C = c
        self.T = t
        self.C_Embedding = None
        self.PA = None
        # self.Q2S = q2s


class QAList:
    def __init__(self):
        self.qa_list = []

    def append_(self, qa):
        self.qa_list.append(qa)

    def remove_qa_of_this_context(self, context):
        for idx, qa in enumerate(self.qa_list):
        #for qa in self.qa_list:
            if context == qa.C:
                #print(qa.id)
                self.qa_list[idx].C = ''
                self.qa_list[idx].C_Embedding = np.zeros(shape=(768,), dtype='float32')
                logger.debug(f"line {qa.id+1} in training-set has the same context with current context:{context} ")

    def set_all_context_embedding(self):
        contexts_all = []
        for qa in self.qa_list:
            contexts_all.append(qa.C)
        #context_all_embedding = word_embedding(contexts_all)

        context_all_embedding = sbert_model.encode(contexts_all)

        for id in range(len(self.qa_list)):
            self.qa_list[id].C_Embedding = context_all_embedding[id]

    def get_all_context_embedding(self):
        context_embedding_list = []
        for qa in self.qa_list:
            context_embedding_list.append(qa.C_Embedding)
        #return np.array(context_embedding_list)  # return np.array()
        print("Embedding count "+str(len(context_embedding_list)))
        return context_embedding_list  # return np.array()


In [ ]:
import time
import os
import copy
import sys
sys.path.append('QAQA-master/scripts') # udpate location
sys.path.append('QAQA-master/benepar_en3/') # udpate location

import pandas as pd

from configparser import ConfigParser
from utils.my_logger import logger
#from utils.read_data import load_qa
#from utils.model_predict import run_predict

from generate.template import *
# from calc_sim.sent_sim import calculate_sim_origin_target, calculate_sim_origin_sentence
from utils.preprocess import get_sentences_from_contexts, context_preprocess
# from calc_sim.answer_sim import is_same_answer


begin_time = time.time()
# # logger.debug(f"Start to run the script {__file__}")
# project_name = sys.argv[1].strip()
# config_file = ''
# if project_name == 'boolq':
#     config_file = 'config_boolq.ini'
# elif project_name == 'squad2':
#     config_file = 'config-squad2.ini'
# elif project_name == 'narrative':
#     config_file = 'config-narrative.ini'
# else:
#     assert False, "No such project"

config = ConfigParser()
config.read('/QAQA-master/scripts/config/config-narrative.ini') # update location
params = config['PARAMETERS']

data_dir = '/content/'
training_set_file = data_dir + 'so_nltk_spacy_data_similar_answers_qaqa_input.csv' #params['TRAINING_FILE_PATH']
test_set_file = training_set_file #params['TEST_FILE_PATH']

# qa_model_path = params['QA_MODEL_PATH']
attack_mods = ['EQ','EQC','ETI'] #params['ATTACK_MOD'].strip().split(',')
extra_sent_num2context = int(params['EXTRA_NUM2C'].strip())
max_attack_num = int(params['MAX_ATTACK_NUM'].strip())
combined_context_num = int(params['CONTEXT_NUM'].strip())
origin_answer_flag = params['ORIGIN_ANSWER']
print(attack_mods)

['EQ', 'EQC', 'ETI']


In [ ]:

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

def write_to_file(df, uid, attack_type, new_question, new_context, new_answer, new_predict_res, is_bug):
  global file_comprehensive_res

  df_out_row = df[df['uid'] == uid]

  df_out_row['attack_type'] = attack_type.strip()
  df_out_row['new_question'] = new_question.strip()
  df_out_row['new_context'] = new_context.strip()
  df_out_row['new_answer'] = new_answer.strip()
  # df_out_row['new_predict_res'] = new_predict_res.strip()
  # df_out_row['is_bug'] = is_bug

  df_out = pd.read_csv(file_comprehensive_res, encoding='utf-8')
  df_out = df_out.append(df_out_row, ignore_index=True)

  df_out.to_csv(file_comprehensive_res, index=False, encoding='utf-8')


In [ ]:
import time
import os
import copy
import sys
sys.path.append('/QAQA-master/scripts') # update location
sys.path.append('/QAQA-master/benepar_en3/')  # update location

import pandas as pd

from configparser import ConfigParser
from utils.my_logger import logger
#from utils.read_data import load_qa
#from utils.model_predict import run_predict

from generate.template import *
# from calc_sim.sent_sim import calculate_sim_origin_target, calculate_sim_origin_sentence
from utils.preprocess import get_sentences_from_contexts, context_preprocess
# from calc_sim.answer_sim import is_same_answer


begin_time = time.time()
# # logger.debug(f"Start to run the script {__file__}")
# project_name = sys.argv[1].strip()
# config_file = ''
# if project_name == 'boolq':
#     config_file = 'config_boolq.ini'
# elif project_name == 'squad2':
#     config_file = 'config-squad2.ini'
# elif project_name == 'narrative':
#     config_file = 'config-narrative.ini'
# else:
#     assert False, "No such project"

config = ConfigParser()
config.read('/QAQA-master/scripts/config/config-narrative.ini')  # update location
params = config['PARAMETERS']

data_dir = '/content/'
training_set_file = data_dir + 'so_nltk_spacy_data_similar_answers_qaqa_input.csv' #params['TRAINING_FILE_PATH']
test_set_file = training_set_file #params['TEST_FILE_PATH']

# qa_model_path = params['QA_MODEL_PATH']
attack_mods = params['ATTACK_MOD'].strip().split(',')
attack_mods = ['EQ','EQC','ETI']
extra_sent_num2context = int(params['EXTRA_NUM2C'].strip())
max_attack_num = int(params['MAX_ATTACK_NUM'].strip())
combined_context_num = int(params['CONTEXT_NUM'].strip())
origin_answer_flag = params['ORIGIN_ANSWER']
# ------------------------------------------------------------------------------------------------------------------
project_name = test_set_file.split("data_")[-1].split('_')[0]
logger.debug(f"project name: {project_name}")
if project_name != "boolq" and "TI" in attack_mods:
    attack_mods.remove("TI")
is_boolq = False
if project_name == 'boolq':
    is_boolq = True

res_dir = data_dir # f'../results/{project_name}/res-dev'
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
file_comprehensive_res = data_dir + 'so_nltk_spacy_data_similar_answers_qaqa_applied_Questions_Only.csv' # os.path.join(res_dir, "so_nltk_spacy_data_similar_answers_qaqa_test_cases_all.csv")
# file_comprehensive_res_bug = os.path.join(res_dir, "so_nltk_spacy_data_similar_answers_qaqa_violation_all.csv")

# ----------------------------load training set and development set ------------------------------------------------

data = pd.read_csv(test_set_file, encoding='utf-8')
df_out = pd.read_csv(file_comprehensive_res, encoding='utf-8')

qa_list_obj_test = load_qa(test_set_file, end_id=2000)
logger.debug("origin question number: " + str(len(qa_list_obj_test.qa_list)))
qa_list_obj_training = load_qa(training_set_file, end_id=2000, remove_redundancy_context=True)

# with open(f"../datasets/compress/{project_name}.comp.tsv", "r", encoding="utf-8") as f:
compress_question_list = data['question']

for obj in qa_list_obj_test.qa_list:
    is_bug = False
    new_question = ''
    new_context = ''
    new_answer = ''
    new_predict_res = ''

    existing_id = df_out[df_out['uid'] == obj.uid]['uid'].tolist()
    if len(existing_id) > 0:
      continue


    this_attack = random.choice(attack_mods)
    logger.info(f"The qa id[test-set] is: {obj.id}")
    if origin_answer_flag == 'SUT':
        ori_ans =  obj.A # AUTHOR Change# run_predict(obj.Q + '\n ' + '('+obj.T+') ' + obj.C)[0]
    elif origin_answer_flag == 'GT':
        ori_ans = obj.A
    else:
        raise Exception(print("wrong key of origin_answer_flag!"))
    # ----------calculate the similarity between question and each context------------------------------------------
    # -------ignore the qa[in train-set] that have same context with current context.-------------------------------
    qa_list_obj_training_no_this = copy.deepcopy(qa_list_obj_training)
    qa_list_obj_training_no_this.remove_qa_of_this_context(obj.C)
    # calculate sim between 'question' in dev-set and 'context' in train-set!   return [(id,sim_value)]
    # compress question into simple question.
    temp_line_list = compress_question_list[obj.id].split("\t")
    if not obj.Q.strip().startswith(temp_line_list[0].strip()):
        logger.warn("check is same: " + obj.Q + ", " + temp_line_list[0])
    compress_qu = temp_line_list[-1].strip() + "?"

    top_n_sim_contexts_list = calculate_sim_origin_target(compress_qu, qa_list_obj_training_no_this, combined_context_num)
    # print("top_n_sim_contexts_list:", top_n_sim_contexts_list)
    # ------------------------ split top-n similarity contexts into a sentences list. ------------------------------
    extra_sentences_list = []
    sentence_qa_id_dict = {}
    for count in range(combined_context_num):
        if count >= len(top_n_sim_contexts_list):
          break
        context_qa_id = top_n_sim_contexts_list[count][0]
        this_qa = qa_list_obj_training_no_this.qa_list[context_qa_id]
        this_sentences_list = get_sentences_from_contexts([this_qa.C])  # eliminate coreference in this process.
        extra_sentences_list.extend(this_sentences_list)
        for s in this_sentences_list:
            sentence_qa_id_dict[s] = context_qa_id

    # ------------calculate the similarity between question and each sentence from context--------------------------
    top_N_sim_sentences_list = calculate_sim_origin_sentence(compress_qu, extra_sentences_list,
                                                              top_n=max_attack_num+extra_sent_num2context-1)
    # top_N_sim_sentences_list = list(np.array(top_N_sim_sentences_list_)[:, 0])
    logger.debug("..................................................................")
    logger.debug(f"origin question is: {obj.Q}")
    logger.debug(f"compress question is: {compress_qu}")
    logger.debug("top_N_sim_sentences_list" + str(top_N_sim_sentences_list))  # [(sentence, sim)]

    for attack_times in range(max_attack_num):
        # --------------------------------generate a new input(question, context) and answer------------------------
        selected_sent = top_N_sim_sentences_list[attack_times][0]
        selected_sent_processed = context_preprocess(selected_sent)  # remove bracket.
        selected_sent_next = top_N_sim_sentences_list[attack_times+1][0]
        sim_question_sent = top_N_sim_sentences_list[attack_times][1]

        # for fear the selected question is too short(e.g., only contain a single word)
        if len(selected_sent_processed) < 10:
            selected_sent_processed = selected_sent_next
        # --------------------combine multi sent to one that will be insert into origin contexts--------------------
        selected_sent_combine = ''
        for i in range(extra_sent_num2context):
            if attack_times + i < len(top_N_sim_sentences_list):  # for fear index out of boundary
                selected_sent_combine += (top_N_sim_sentences_list[attack_times + i][0] + ' ')
            else:
                selected_sent_combine = selected_sent_processed
                logger.warn("[Warning!] [index out of scope] the hyper-parameter 'CONTEXT_NUM' is too small! ")

        if 'EC' == this_attack:
            new_context = add_extra2context(obj.C, selected_sent_combine)
            new_question = obj.Q
            # new_predict_res = run_predict(new_question + '\n '+new_context)[0]  # ('+obj.T+')
            # if not is_same_answer(new_predict_res, ori_ans, is_bool=is_boolq):
            #     logger.debug('.....................A Successful EC Attack...................................\n')
            #     logger.debug(f"Question: {new_question}")
            #     logger.debug(f"Origin context: {obj.C}")
            #     logger.debug(f"New context: {new_context}")
            #     logger.debug(f"Ground truth:{obj.A}, New answer:{new_predict_res}")
            #     logger.debug('...................................................................................\n')
            #     is_bug = True
                # str_output_bug = f"{obj.id}\t{new_question} \\n {new_context}\t{ori_ans}->{new_predict_res}\tEC\n"
                # with open(file_comprehensive_res_bug, 'a', encoding='utf-8') as f:
                #     f.write(str_output_bug)
            # str_output = f"{new_question} \\n {new_context}\t{ori_ans}\t{new_predict_res}\n"
            # with open(file_comprehensive_res, 'a', encoding='utf-8') as f:
            #     f.write(str_output)
            write_to_file(data, obj.uid, this_attack, new_question, new_context, new_answer, new_predict_res, is_bug)

        if 'EQ' == this_attack:
            new_question = obj.Q
            if project_name == 'boolq' and random.randint(0, 1) == 1:
                new_question = negative_question(new_question)
            new_question = add_extra2question(new_question, selected_sent_processed)
            # new_predict_res = run_predict(new_question + '\n '+obj.C)[0]  # ('+obj.T+')
            # if not is_same_answer(new_predict_res, ori_ans, is_bool=is_boolq):
            #     logger.debug('.....................A Successful EQ Attack...................................\n')
            #     logger.debug(f"Origin question: {obj.Q}")
            #     logger.debug(f"New question: {new_question}")
            #     logger.debug(f"Context: {obj.C}")
            #     logger.debug(f"Ground truth:{ori_ans}, Predicting answer:{new_predict_res}")
            #     logger.debug('...................................................................................\n')
                # str_output_bug = f"{obj.id}\t{new_question} \\n {obj.C}\t{ori_ans}->{new_predict_res}\tEQ\n"
                # with open(file_comprehensive_res_bug, 'a', encoding='utf-8') as f:
                #     f.write(str_output_bug)
            # str_output = f"{new_question} \\n {obj.C}\t{ori_ans}\t{new_predict_res}\n"
            # with open(file_comprehensive_res, 'a', encoding='utf-8') as f:
            #     f.write(str_output)
            write_to_file(data, obj.uid, this_attack, new_question, new_context, new_answer, new_predict_res, is_bug)


        if 'EQC' == this_attack:
            new_question = obj.Q
            if project_name == 'boolq' and random.randint(0, 1) == 1:   # [WARNING] add negative attack!!!
                new_question = negative_question(new_question)
            new_question = add_extra2question(new_question, selected_sent_processed)
            #AUTHOR DISABLED IT: new_context = add_extra2context(obj.C, selected_sent_combine if ori_ans == 'no' else selected_sent_next)
            # new_predict_res = run_predict(new_question + '\n '+new_context)[0]   # ('+obj.T+')
            # if not is_same_answer(new_predict_res, ori_ans, is_bool=is_boolq):
            #     logger.debug('.....................A Successful  EQC Attack.................................\n')
            #     logger.debug(f"New question: {new_question}")
            #     logger.debug(f"New context: {new_context}")
            #     logger.debug(f"Ground truth:{ori_ans}, Predicting answer:{new_predict_res}")
            #     logger.debug('...................................................................................\n')
                # str_output_bug = f"{obj.id}\t{new_question} \\n {new_context}\t{ori_ans}->{new_predict_res}\tEQC\n"
                # with open(file_comprehensive_res_bug, 'a', encoding='utf-8') as f:
                #     f.write(str_output_bug)
            # str_output = f"{new_question} \\n {new_context}\t{ori_ans}\t{new_predict_res}\n"
            # with open(file_comprehensive_res, 'a', encoding='utf-8') as f:
            #     f.write(str_output)
            write_to_file(data, obj.uid, this_attack, new_question, new_context, new_answer, new_predict_res, is_bug)

        if 'ETI' == this_attack:
            selected_qa_id = sentence_qa_id_dict[selected_sent]
            selected_qa = qa_list_obj_training_no_this.qa_list[selected_qa_id]
            if project_name == "boolq":
                new_question, new_context = add_input_as_redundancy(obj, selected_qa)
            else:
                new_question, new_context = add_wh_question_as_redundancy(obj, selected_qa)
            # new_predict_res = run_predict(new_question + '\n '+new_context)[0]  # ('+obj.T+' && '+selected_qa.T+')
            # if not is_same_answer(new_predict_res, ori_ans, is_bool=is_boolq):
            #     logger.debug('....................A Successful ETI Attack........................\n')
            #     logger.debug(f"Origin question: {obj.Q}")
            #     logger.debug(f"Selected question: {selected_qa.Q}")
            #     logger.debug(f"New Question: {new_question}")
            #     logger.debug(f"Origin context: {obj.C}")
            #     logger.debug(f"Selected context: {selected_qa.C}")
            #     logger.debug(f"New context: {new_context}")
            #     logger.debug(f"Ground truth:{ori_ans}, Predicting answer:{new_predict_res}")
            #     logger.debug('...................................................................................\n')
                # str_output_bug = f"{obj.id}\t{new_question} \\n {new_context}\t{ori_ans}->{new_predict_res}\tETI\n"
                # with open(file_comprehensive_res_bug, 'a', encoding='utf-8') as f:
                #     f.write(str_output_bug)
            # str_output = f"{new_question} \\n {new_context}\t{ori_ans}\t{new_predict_res}\n"
            # with open(file_comprehensive_res, 'a', encoding='utf-8') as f:
            #     f.write(str_output)
            write_to_file(data, obj.uid, this_attack, new_question, '', new_answer, new_predict_res, is_bug) # AUTHOR: DO NOT WRITE CONTEXT

        if 'TI' == this_attack:
            selected_qa_id = sentence_qa_id_dict[selected_sent]
            selected_qa = qa_list_obj_training_no_this.qa_list[selected_qa_id]
            ori_ans2 = selected_qa.A if origin_answer_flag == 'GT' \
                else run_predict(selected_qa.Q + '\n (' + selected_qa.T + ')' + selected_qa.C)[0]

            new_question, new_context, new_answer = combine2input(obj, selected_qa, ori_ans, ori_ans2)
            # new_predict_res = run_predict(new_question + '\n '+new_context)[0]  # ('+obj.T+' && '+selected_qa.T+')
            # if not is_same_answer(new_predict_res, new_answer, is_bool=is_boolq):
            #     logger.debug('.....................A Successful TI Attack............................\n')
            #     logger.debug(f"Origin question: {obj.Q}")
            #     logger.debug(f"Selected question: {selected_qa.Q}")
            #     logger.debug(f"New Question: {new_question}")
            #     logger.debug(f"Origin context: {obj.C}")
            #     logger.debug(f"Selected context: {selected_qa.C}")
            #     logger.debug(f"New context: {new_context}")
            #     logger.debug(f"Ground truth:{ori_ans}+{ori_ans2}={new_answer}, Predicting answer:{new_predict_res}")
            #     logger.debug('...................................................................................\n')
                # str_output_bug = f"{obj.id}\t{new_question} \\n {new_context}\t{new_answer}->{new_predict_res}\tTI\n"
                # with open(file_comprehensive_res_bug, 'a', encoding='utf-8') as f:
                #     f.write(str_output_bug)
            # str_output = f"{new_question} \\n {new_context}\t{new_answer}\t{new_predict_res}\n"
            # with open(file_comprehensive_res, 'a', encoding='utf-8') as f:
            #     f.write(str_output)

            write_to_file(data, obj.uid, this_attack, new_question, new_context, new_answer, new_predict_res, is_bug)

    #     break
    # break



end_time = time.time()
logger.debug(f"Finish question_gen for question_{obj.id},during time is {round(end_time - begin_time, 2)}s")
